In [226]:
import pandas as pd
import re
from datetime import datetime, timedelta

In [227]:
earthquakes=pd.read_csv('./VillarricaEarthquakes.csv')

In [228]:
eruptions=pd.read_csv('./VillarricaEruptions.csv')

# Exploring datasets

In [229]:
eruptions.head()

,volcano_number,volcano_name,eruption_number,eruption_category,vei,start_year,start_month,start_day,latitude,longitude,date
0,357120,Villarrica,12184,Confirmed Eruption,1.0,2009.0,11.0,22.0,-39.42,-71.93,2009-11-22
1,357120,Villarrica,12183,Confirmed Eruption,1.0,2009.0,1.0,29.0,-39.42,-71.93,2009-01-29
2,357120,Villarrica,12182,Confirmed Eruption,1.0,2008.0,10.0,26.0,-39.42,-71.93,2008-10-26
3,357120,Villarrica,12181,Confirmed Eruption,1.0,2004.0,8.0,5.0,-39.42,-71.93,2004-08-05
4,357120,Villarrica,12180,Confirmed Eruption,1.0,2003.0,5.0,23.0,-39.42,-71.93,2003-05-23


In [230]:
eruptions=eruptions.rename(columns={'latitude':'volcano_latitude','longitude': 'volcano_longitude'})

In [231]:
eruptions['date']=pd.to_datetime(eruptions['date'],format="%Y/%m/%d")

In [232]:
eruptions['Days since last eruption']=eruptions['date'].diff(periods=-1).dt.days

In [233]:
eruptions.loc[eruptions['eruption_number']==12150,'Days since last eruption']=700

In [234]:
earthquakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726 entries, 0 to 1725
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         1726 non-null   object 
 1   mag        1726 non-null   float64
 2   longitude  1726 non-null   float64
 3   latitude   1726 non-null   float64
 4   date       1726 non-null   object 
dtypes: float64(3), object(2)
memory usage: 67.5+ KB


In [235]:
earthquakes['date']=pd.to_datetime(earthquakes['date'],format="%Y/%m/%d")

# Merging datasets

In [246]:
for day in range(0,31):    
    earthquakes[f'date-{day}']=earthquakes['date']-timedelta(days=day)

In [248]:
for x in range(0, 31):
    globals()['villarrica%s' % x] = earthquakes.join(eruptions.set_index('date'), on=f'date-{x}')
    globals()['villarrica%s' % x] = globals()['villarrica%s' % x].dropna()

In [252]:
villarrica=pd.DataFrame()
for i in range(0,31):
    villarrica=pd.concat([villarrica, globals()['villarrica%s' % i]])

In [254]:
villarrica0=earthquakes.join(eruptions.set_index('date'), on='date-0')

In [255]:
villarrica=pd.concat([villarrica,villarrica0])

In [259]:
villarrica=villarrica.drop_duplicates()

# Final adjustments

In [286]:
villarrica

,id,mag,longitude,latitude,date,date-0,date-1,date-2,date-3,date-4,...,eruption_number,eruption_category,vei,start_year,start_month,start_day,volcano_latitude,volcano_longitude,Days since last eruption,was_erupted
0,usp000hrgh,4.0,-73.333,-42.262,2010-12-22,2010-12-22,2010-12-21,2010-12-20,2010-12-19,2010-12-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0
1,usp000hr5d,4.5,-73.235,-36.492,2010-12-19,2010-12-19,2010-12-18,2010-12-17,2010-12-16,2010-12-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0
2,usp000hr1v,4.3,-73.726,-37.722,2010-12-17,2010-12-17,2010-12-16,2010-12-15,2010-12-14,2010-12-13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0
3,usp000hqxc,4.5,-72.108,-40.491,2010-12-14,2010-12-14,2010-12-13,2010-12-12,2010-12-11,2010-12-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0
4,usp000hqtn,4.7,-73.529,-37.045,2010-12-11,2010-12-11,2010-12-10,2010-12-09,2010-12-08,2010-12-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721,iscgem893991,6.2,-73.519,-38.589,1951-01-03,1951-01-03,1951-01-02,1951-01-01,1950-12-31,1950-12-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,623.0,0.0
1722,iscgem896480,6.9,-72.676,-37.665,1949-04-20,1949-04-20,1949-04-19,1949-04-18,1949-04-17,1949-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3113.0,0.0
1723,iscgem901645,7.0,-73.630,-42.041,1940-10-11,1940-10-11,1940-10-10,1940-10-09,1940-10-08,1940-10-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,625.0,0.0
1724,iscgem901785,7.8,-72.315,-36.305,1939-01-25,1939-01-25,1939-01-24,1939-01-23,1939-01-22,1939-01-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1791.0,0.0


In [267]:
villarrica['was_erupted']=villarrica['volcano_number'].fillna(0)

In [270]:
villarrica['was_erupted'].loc[villarrica['was_erupted']==357120]=1

In [285]:
villarrica=villarrica.sort_values('date',ascending=False)

In [282]:
villarrica['Days since last eruption']=villarrica['date'].diff(periods=-1).dt.days

In [283]:
villarrica['Days since last eruption']

0          3.0
1          2.0
2          3.0
3          3.0
4          4.0
         ...  
1721     623.0
1722    3113.0
1723     625.0
1724    1791.0
1725       NaN
Name: Days since last eruption, Length: 1760, dtype: float64

In [287]:
villarrica.head(5)

,id,mag,longitude,latitude,date,date-0,date-1,date-2,date-3,date-4,...,eruption_number,eruption_category,vei,start_year,start_month,start_day,volcano_latitude,volcano_longitude,Days since last eruption,was_erupted
0,usp000hrgh,4.0,-73.333,-42.262,2010-12-22,2010-12-22,2010-12-21,2010-12-20,2010-12-19,2010-12-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0
1,usp000hr5d,4.5,-73.235,-36.492,2010-12-19,2010-12-19,2010-12-18,2010-12-17,2010-12-16,2010-12-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0
2,usp000hr1v,4.3,-73.726,-37.722,2010-12-17,2010-12-17,2010-12-16,2010-12-15,2010-12-14,2010-12-13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0
3,usp000hqxc,4.5,-72.108,-40.491,2010-12-14,2010-12-14,2010-12-13,2010-12-12,2010-12-11,2010-12-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0
4,usp000hqtn,4.7,-73.529,-37.045,2010-12-11,2010-12-11,2010-12-10,2010-12-09,2010-12-08,2010-12-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0


In [288]:
for i in villarrica.index:
    villarrica.loc[villarrica.index==i,'Days since last eruption']

Int64Index([   0,    1,    2,    3,    4,    5,    6,    8,    7,    9,
            ...
            1716, 1717, 1718, 1719, 1720, 1721, 1722, 1723, 1724, 1725],
           dtype='int64', length=1760)